# Use SMILES corrector for exploration

This version demonstrates how to create analogues for desired molecule. It shows how to introduce errors into the molecule, initialize the SMILES corrector & fix the errors

# Setup

In [ ]:
import torch
import os

import pandas as pd
import numpy as np

import random

import pickle

from src.invalidSMILES import get_invalid_smiles
from src.preprocess import standardization_pipeline, remove_smiles_duplicates
from src.modelling import initialize_model, correct_SMILES

# set random seed, used for error generation & initiation transformer
SEED = 42
random.seed(SEED)

# Introduce errors into molecule or set of molecules

In [ ]:
# file name
folder_out = "Data/"
name = 'selective_ki'
# number of errorneous sequences to generate per molecule
errors_per_molecule = 10
# where to save the errors
if not os.path.exists(f"Data/explore"):
    os.makedirs(f"Data/explore")
error_source = "Data/explore/%s_with_%s_errors_index.csv" % (
    name, errors_per_molecule)

In [ ]:
# standardize SMILES
standardize = False
if standardize:
    df = pd.read_csv('%s%s.csv' % (folder_out, name), usecols=['SMILES']).dropna()
    df["STD_SMILES"] = df.apply(
        lambda row: standardization_pipeline(row["SMILES"]),
        axis=1).dropna()
    df = df.drop(columns=['SMILES'])
    df.to_csv('%s%s.csv' % (folder_out, name), index=None)
else:
    df = pd.read_csv('%s%s.csv' % (folder_out, name),
                        usecols=['STD_SMILES']).dropna()

In [ ]:
folder_raw = "RawData/"

df_frag = pd.read_csv(f"{folder_raw}gbd_8.csv",
                        names=["FRAGMENT"],
                        usecols=[0],
                        header=0).dropna()

# duplicate SMILES to create multiple errors per molecule
df = pd.concat([df] * (errors_per_molecule), ignore_index=False)
index_list = list(df.index.values)
smiles = list(df['STD_SMILES'].values)
df = get_invalid_smiles(df,
                        df_frag,
                        SEED,
                        invalid_type="all",
                        num_errors=1)
df = df.drop(columns=['FRAGMENT']).reset_index(drop=True)
df = df.rename(columns={"ERROR": "SMILES"})
df['SMILES_TARGET'] = df["SMILES"]
df.index = index_list
df['ORIGINAL_SMILES'] = smiles
# drop duplicate erroneous SMILES
df = df.drop_duplicates(subset=['SMILES'])
df.to_csv(error_source)

# Initialize model 

In [ ]:
# model parameters
folder_out = "Data/"
invalid_type = 'multiple'
num_errors = 12
threshold = 200
data_source = f"PAPYRUS_{threshold}"
gpu = '1'

# define this in test.py
os.environ["CUDA_VISIBLE_DEVICES"] = gpu
initialize_source = 'Data/papyrus_rnn_S.csv'

#device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, out, SRC = initialize_model(folder_out,
                                    data_source,
                                    error_source=initialize_source,
                                    device=device,
                                    threshold=threshold,
                                    epochs=30,
                                    layers=3,
                                    batch_size=16,
                                    invalid_type=invalid_type,
                                    num_errors=num_errors)
print(out)

# Correct SMILES

In [ ]:
valids, df_output = correct_SMILES(model, out, error_source, device,
                                    SRC)
df_output.to_csv(
    f"Data/explore/{error_source.split('/')[2].split('.')[0]}_fixed.csv",
    index=False)

# keep unique sequences
df_output = df_output.dropna(subset = ["CORRECT"])
df_output["STD_SMILES"] = df_output.apply(
            lambda row: standardization_pipeline(row["CORRECT"]), axis=1
        ).dropna()
df_output = remove_smiles_duplicates(df_output, subset="STD_SMILES")

df_output.to_csv(
    f"Data/explore/{error_source.split('/')[2].split('.')[0]}_fixed_unique.csv",
    index=False)